In [416]:
import matplotlib.pyplot as plt
import networkx as nx
import os

import folium
from MRD.mrd_package.mrd.Data import Data

# 中心座標
center = 35.667183, 139.738408
# 距離
distance = 0.5 # km
# データディレクトリ
data_dir = os.path.normpath(r"C:\Users\hiroki\PycharmProjects\mrd\MRD\data")

d = Data()
d.set_area(center, distance)
d.set_path(data_dir)
d.load()
g = nx.DiGraph()

1/2 Found pickle file (533935.pickle). loading..
2/2 Found pickle file (533945.pickle). loading..


In [417]:
g.clear()

# ノードを追加する
nodes = d.road.all_node

for k, v in nodes.items():
    g.add_node(k, pos=(v.lat, v.lon))

# エッジを追加する 距離はweight, 細かい緯度経度はlink属性に
links = d.road.all_way
edges = []

for k, v in links.items():
    if v.link_kisei in (4, 6): # 一方通行（正方向）
        edges.append([k[0], k[1], {'weight':v.link_length, 'link':v.link} ])
    elif v.link_kisei in (5, 7): # 一方通行（逆方向）
        edges.append([k[1], k[0], {'weight':v.link_length, 'link':v.link[::-1]} ])
    else:
        edges.append([k[0], k[1], {'weight':v.link_length, 'link':v.link} ])
        edges.append([k[1], k[0], {'weight':v.link_length, 'link':v.link[::-1]} ])
    g.add_edges_from(edges)

In [418]:
# 最短経路
r = nx.shortest_path(g, source=53393599162, target=53393578379   )

In [419]:
r # 経路ノード

[53393599162, 53393503208, 53393502334, 53393514817, 53393578386, 53393578379]

In [420]:
l = []  # 経路ノードから緯度経度を取得する
for i in range(len(r)-1):
    uv = (r[i], r[i+1])
    l.extend(g.edges[uv]['link'])
l

[(35.66718981616313, 139.73841888111238),
 (35.66697318108486, 139.73865636971516),
 (35.66697318108486, 139.73865636971516),
 (35.66733148445288, 139.73909381820457),
 (35.66733148445288, 139.73909381820457),
 (35.667481470349365, 139.73921880156874),
 (35.66778144348389, 139.73951876405974),
 (35.66778144348389, 139.73951876405974),
 (35.667914754256245, 139.73925628093247),
 (35.66799807319546, 139.73908129240482),
 (35.66799807319546, 139.73908129240482),
 (35.66777309435089, 139.7388938173589)]

In [421]:
# 地図
map_ = folium.Map(location=[center[0], center[1]], zoom_start=18, control_scale=True, prefer_canvas=True)
folium.PolyLine(locations=l, color="red", weight="4.5", opacity=0.6).add_to(map_)

In [422]:
map_